In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import pandas as pd
import numpy as np

In [6]:
datasets_path = '/home/kuptservol/.datasets/kaggle'

In [ ]:
! kaggle competitions download -c dmia-production-ml-2021-1-passwords -p {datasets_path}

In [7]:
data_path = datasets_path + '/dmia-production-ml-2021-1-passwords'

In [ ]:
! mkdir {data_path}

In [ ]:
! unzip {datasets_path}/dmia-production-ml-2021-1-passwords.zip -d {data_path}

In [ ]:
! ls {data_path}

In [290]:
train_df = pd.read_csv(data_path+'/train.csv/train.csv')

In [291]:
train_df.head(10)

,Password,Times
0,631XniVx2lS5I,2
1,LEGIT747,1
2,742364es,1
3,3846696477,1
4,laurahop,2
5,130185m,1
6,EzjNQB5j,1
7,mez2288,1
8,distopia,1
9,05031995m,1


In [292]:
train_df.sort_values(by='Times', ascending=False).head(20)

,Password,Times
2715397,123456,55893
3136279,qwerty,13137
1175081,123456789,11696
2363307,12345,10938
2988373,1234,6432
2307329,111111,5682
3348280,1234567,4796
1795496,dragon,3927
3336606,123123,3845
3798071,baseball,3565


In [293]:
train_df.describe()

,Times
count,4.151496e+06
mean,1.922133e+00
std,3.250090e+01
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,1.000000e+00
max,5.589300e+04


In [294]:
def words_to_char(inputs):
    characters=[]
    for i in inputs:
        characters.append(i)
    return characters

In [295]:
len(train_df)

4151496

In [296]:
train_df.fillna('', inplace=True)

In [297]:
X = train_df['Password']

In [298]:
y = train_df['Times']

In [299]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=words_to_char)
vectorizer.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function words_to_char at 0x7f5983d898c8>,
                use_idf=True, vocabulary=None)

In [300]:
from scipy.sparse import csr_matrix, vstack, hstack, coo_matrix

In [301]:
train_df['Password'][:10].map(lambda x:x)

0    631XniVx2lS5I
1         LEGIT747
2         742364es
3       3846696477
4         laurahop
5          130185m
6         EzjNQB5j
7          mez2288
8         distopia
9        05031995m
Name: Password, dtype: object

In [302]:
def count_digit(password):
    c = 0
    for p in password:
        if p.isnumeric():
            c += 1
    return c

In [303]:
def count_char(password):
    c = 0
    for p in password:
        if p.isalpha():
            c += 1
    return c

In [304]:
def only_digits_or_only_nums(password):
    chars = count_digit(password)
    digits = count_char(password)
    c = 1 if chars == 0 or digits == 0 else 0 
    return c

In [305]:
import re
special_characters = re.compile('[@_!#$%^&*()<>?/\|}{~:]')
def count_special_symbol(password):
    c = 0
    for p in password:
        if special_characters.search(p) is not None:
            c += 1
    return c

In [103]:
!kaggle datasets download -d 'rtatman/english-word-frequency' -p ~/.datasets/english-word-frequency

 47%|█████████████████▊                    | 1.00M/2.13M [00:00<00:00, 6.59MB/s]
100%|██████████████████████████████████████| 2.13M/2.13M [00:00<00:00, 11.8MB/s]


In [306]:
word_freq_df = pd.read_csv('~/.datasets/english-word-frequency/unigram_freq.csv')

In [307]:
word_freq_df.head()

,word,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698


In [308]:
word_freq = {wf[0]:wf[1] for wf in word_freq_df.values}

In [309]:
def count_word_freq(password):
    if password in word_freq:
        return word_freq[password]
    else:
        return 0

In [310]:
def num_uniq_symb(password):
    return len(set((p for p in password)))

In [311]:
X = coo_matrix()

TypeError: __init__() missing 1 required positional argument: 'arg1'

In [ ]:
def get_features(df):
    df.fillna('', inplace=True)
    X = vectorizer.transform(df['Password'])
    print('vectorizer stage')
    df['PasswordLen']=df['Password'].map(lambda x:len(x))
    print('PasswordLen stage')
    df['DigitsNum']=df['Password'].map(lambda x:count_digit(x))
    print('DigitsNum stage')
    df['CharNum']=df['Password'].map(lambda x:count_char(x))
    print('CharNum stage')
    df['SpecialSymbolNum']=df['Password'].map(lambda x:count_special_symbol(x))
    print('SpecialSymbolNum stage')
    df['WorsFreq']=df['Password'].map(lambda x:count_word_freq(x))
    print('WorsFreq stage')
    df['NumUniqSymb']=df['Password'].map(lambda x:num_uniq_symb(x))
    print('NumUniqSymb stage')
    X = hstack([X, coo_matrix(df['PasswordLen'].values).T])
#     X = coo_matrix(df['PasswordLen'].values).T
    X = hstack([X, coo_matrix(df['DigitsNum'].values).T])
    X = hstack([X, coo_matrix(df['SpecialSymbolNum'].values).T])
    X = hstack([X, coo_matrix(df['WorsFreq'].values).T]) 
    X = hstack([X, coo_matrix(df['NumUniqSymb'].values).T]) 
    return X

In [ ]:
X = get_features(train_df)

In [ ]:
train_df.sort_values(by='Times', ascending=False).head(10)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression(normalize=True)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.coef_

In [ ]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
def convert(predicted):
    return list(map(lambda x: int(abs(x)), predicted))

In [ ]:
mean_squared_log_error(convert(model.predict(X_test)), y_test)

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV

In [312]:
# alphas = np.arange(1, 100, 10)
alphas = [.0001, .001,.01,.1, 2, 5, 10, 20, 30, 50, 100, 1000, 10000]
# model = RidgeCV(alphas=alphas, cv=3, normalize=True)
model = LassoCV(alphas=alphas, cv=3, normalize=True)

In [313]:
model.fit(X_train, y_train)

LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 2, 5, 10, 20, 30, 50, 100, 1000,
                10000],
        copy_X=True, cv=3, eps=0.001, fit_intercept=True, max_iter=1000,
        n_alphas=100, n_jobs=None, normalize=True, positive=False,
        precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
        verbose=False)

In [314]:
mean_squared_log_error(convert(model.predict(X_test)), y_test)

0.18027505056485285

## Lin reg on grid search

In [334]:
from sklearn.pipeline import Pipeline

In [344]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [345]:
from sklearn import linear_model
regressor = linear_model.SGDRegressor()

In [346]:
pipeline = Pipeline(steps = [('scaling', scaler), ('regression', regressor)])

In [347]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scaling', 'regression', 'scaling__copy', 'scaling__with_mean', 'scaling__with_std', 'regression__alpha', 'regression__average', 'regression__early_stopping', 'regression__epsilon', 'regression__eta0', 'regression__fit_intercept', 'regression__l1_ratio', 'regression__learning_rate', 'regression__loss', 'regression__max_iter', 'regression__n_iter_no_change', 'regression__penalty', 'regression__power_t', 'regression__random_state', 'regression__shuffle', 'regression__tol', 'regression__validation_fraction', 'regression__verbose', 'regression__warm_start'])

In [366]:
parameters_grid = {
    'regression__loss' : ['huber', 'epsilon_insensitive', 'squared_loss'],
    'regression__max_iter' : [3, 5, 10, 50], 
    'regression__penalty' : ['l1', 'l2', 'none'],
    'regression__alpha' : [0.0001, 0.01],
    'scaling__with_mean' : [0., 0.5],
}

In [374]:
from sklearn import model_selection

In [375]:
grid_cv = model_selection.GridSearchCV(pipeline, parameters_grid, scoring = 'neg_mean_squared_error', cv = 4)

In [376]:
import warnings
warnings.filterwarnings('ignore')

In [377]:
X_train_dense = X_train.todense()

In [378]:
%%time
grid_cv.fit(X_train_dense, y_train)

CPU times: user 49min 33s, sys: 9min 16s, total: 58min 50s
Wall time: 48min 16s


GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaling',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('regression',
                                        SGDRegressor(alpha=0.0001,
                                                     average=False,
                                                     early_stopping=False,
                                                     epsilon=0.1, eta0=0.01,
                                                     fit_intercept=True,
                                                     l1_ratio=0.15,
                                                     learning_rate='invscaling',
                                                     loss='squared_loss',
      

In [379]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

-1280.064536339128
{'regression__alpha': 0.01, 'regression__loss': 'squared_loss', 'regression__max_iter': 10, 'regression__penalty': 'l1', 'scaling__with_mean': 0.0}


In [381]:
mean_squared_log_error(convert(grid_cv.best_estimator_.predict(X_test)), y_test)

0.3702082159977019

# RNN/LSTM

In [35]:
import torch
from torch import nn

In [382]:
passwords = train_df['Password'].values

all_chars = set(''.join(passwords))

int2char = dict(enumerate(all_chars))

char2int = {char: ind for ind, char in int2char.items()}

maxlen = len(max(train_df['Password'], key=len))

In [383]:
maxlen

42

In [384]:
X_train, X_test, y_train, y_test = train_test_split(train_df.Password, train_df.Times, test_size=0.2, random_state=42)

In [385]:
X_train_values = X_train.values

padding

In [386]:
for i in range(len(X_train_values)):
  while len(X_train_values[i]) < maxlen:
      X_train_values[i] += ' '

In [387]:
X_train_seq = []

for i in range(len(X_train)):
    X_train_seq.append(X_train_values[i])

In [388]:
X_test_values = X_test.values

padding

In [46]:
for i in range(len(X_test_values)):
  while len(X_test_values[i]) < maxlen:
      X_test_values[i] += ' '

In [389]:
X_test_seq = []

for i in range(len(X_test)):
    X_test_seq.append(X_test_values[i])

In [390]:
char2int[' '] = 100
int2char[100] = ' '

In [391]:
for i in range(len(X_train_values)):
    X_train_seq[i] = [char2int[character] for character in X_train_seq[i]]

In [392]:
for i in range(len(X_test_values)):
    X_test_seq[i] = [char2int[character] for character in X_test_seq[i]]

In [393]:
word_embeddings = nn.Embedding(101, 10)

In [394]:
len(char2int)

98

In [395]:
x_batch, y_batch = next(iter(train_loader))

NameError: name 'train_loader' is not defined

In [396]:
x_batch.shape

NameError: name 'x_batch' is not defined

In [397]:
lstm = nn.LSTM(10, 12, batch_first=True)

In [398]:
linear = nn.Linear(12, 1)

In [399]:
x_batch_l = x_batch.long()

NameError: name 'x_batch' is not defined

In [400]:
class Model(nn.Module):

    def __init__(self, embedding_dim, hidden_dim1, hidden_dim2, vocab_size, tagset_size):
        super(Model, self).__init__()
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim1, batch_first=True)

        self.linear1 = nn.Linear(hidden_dim1, hidden_dim2)

        self.batch_norm = nn.BatchNorm1d(hidden_dim2)

        self.linear2 = nn.Linear(hidden_dim2, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds)
        out = self.linear1(lstm_out[:, -1, :])
        out = self.batch_norm(out)
        out = self.linear2(out)

        return out

In [401]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        # size = 2, 42, 12
        
      
        # out = out.contiguous().view(-1, self.hidden_dim)
        # Take just last rnn cycle out
        out = out[:, -1, :]

        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden.to(device)

In [402]:
from torch.utils.data import DataLoader, TensorDataset

In [403]:
target = torch.Tensor(y_train)
test_target = torch.Tensor(y_test.values)

In [404]:
mse = nn.MSELoss()
eps = 1e-6
# loss = torch.sqrt(criterion(x, y) + eps)
def loss_F(pred, actual):
  return torch.sqrt(mse(torch.log(pred + 1), torch.log(actual + 1)) + eps)

In [ ]:
X_train_input_seq = torch.from_numpy(np.array(X_train_seq)).float()
X_test_input_seq = torch.from_numpy(np.array(X_test_seq)).float()

In [ ]:
subset = 100000

In [ ]:
train_data = TensorDataset(X_train_input_seq[:subset], target[:subset])
test_data = TensorDataset(X_test_input_seq[:subset], test_target[:subset])

In [ ]:
train_data = TensorDataset(X_train_input_seq, target)
test_data = TensorDataset(X_test_input_seq, test_target)

In [ ]:
len(train_data)

In [ ]:
batch_size=1000

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from torch import autograd

In [ ]:
n_epochs = 30
lr = 0.001

# model = Model(input_size=1, output_size=1, hidden_dim=100, n_layers=1)
model = Model(embedding_dim=10, hidden_dim1=50,  hidden_dim2=50, vocab_size=101, tagset_size=1)
model.to(device)

# Define Loss, Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [ ]:
train_losses = []
test_losses = []
iteration_list = []
count = 0
for epoch in range(n_epochs):
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.long().to(device)
        y_batch = y_batch.to(device)

        model.train()
        output = model(x_batch)
        loss = loss_F(output, y_batch)
        train_losses.append(loss.item())
        with autograd.detect_anomaly():
          loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        count += 1

        # if torch.isnan(loss).any():
          # print('Iteration: {}  Loss: {} Grad: {}'.format(count, loss.data, loss.grad))
        if count % 1000 == 0:
          train_losses.append(loss.data)
          iteration_list.append(count)
          print('Iteration: {}  Train Loss: {}'.format(count, loss.data))


    print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
    print("Train Loss: {:.4f}".format(loss.item()), end=' ')
        
    with torch.no_grad():
        for x_test_batch, y_test_batch in test_loader:
            x_test_batch = x_test_batch.long().to(device)
            y_test_batch = y_test_batch.to(device)
            
            model.eval()

            output = model(x_test_batch)
            test_loss = loss_F(output, y_test_batch)
            test_losses.append(test_loss.item())
        # val_loss = np.average(test_losses)
        print("Test Loss: {:.4f}".format(test_loss.data))
        # valid_losses.append(val_loss)

## Submission

In [178]:
submission = pd.read_csv('/home/kuptservol/.datasets/kaggle/dmia-production-ml-2021-1-passwords/sample_submission.csv/sample_submission.csv')

In [179]:
test_df = pd.read_csv('/home/kuptservol/.datasets/kaggle/dmia-production-ml-2021-1-passwords/Xtest.csv/Xtest.csv')

In [180]:
submission.head()

,Id,Times
0,0,1.922133
1,1,1.922133
2,2,1.922133
3,3,1.922133
4,4,1.922133


In [181]:
test_df.head()

,Id,Password
0,0,ThaisCunha
1,1,697775113
2,2,922a16922a
3,3,andy74
4,4,joemack


In [242]:
X_subm = get_features(test_df)

vectorizer stage
PasswordLen stage
DigitsNum stage
CharNum stage
SpecialSymbolNum stage
WorsFreq stage
NumUniqSymb stage


In [243]:
test_preds = model.predict(X_subm)

In [244]:
test_preds[:2]

array([1.93381451, 1.93381451])

In [245]:
test_preds = convert(test_preds)

In [249]:
test_preds[:2]

[1, 1]

In [185]:
submission['Times'] = test_preds

In [186]:
submission['Times'].head()

0    1
1    1
2    1
3    2
4    1
Name: Times, dtype: int64

In [187]:
submission.to_csv(data_path+"/submission.csv",index=False)

In [188]:
! kaggle competitions submit dmia-production-ml-2021-1-passwords -f {data_path}'/submission.csv' -m "My submission"

100%|██████████████████████████████████████| 8.84M/8.84M [00:04<00:00, 2.31MB/s]
Successfully submitted to DMIA.ProductionML 2021.1 Password complexity